In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import spacy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Text cleaning function:

def text_cleaner(text):
    text = re.sub(r'--', ' ', text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [3]:
# Importing and applying cleaning function:

persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

persuasion = re.sub(r'Chapter /d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [4]:
# Prepping the texts to work in spaCy:

nlp = spacy.load('en')

alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Dataframe of the sentences in each text:
sentences = pd.DataFrame(alice_sents + persuasion_sents)

In [5]:
from collections import Counter

def bag_of_ads(text):
    
    ads2 = [token.text
           for token in text
           if token.pos_ == 'ADJ'
           if not token.is_stop]
    
    return [item[0] for item in Counter(ads2).most_common(2000)]

In [6]:
def boa_features(sentences, common_words):
    
        # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to ads, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.text
                 for token in sentence
                 if (not token.is_stop
                     and token.text in common_words)]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [7]:
# Getting the most common adjectives for each text:
alice_ads = bag_of_ads(alice_doc)
persuasion_ads = bag_of_ads(persuasion_doc)

# Combining theit counts to avoid duplicates:
alice_persuasion_ads = set(alice_ads + persuasion_ads)

# Getting the count per sentence and making a dataframe:
ads_df = boa_features(sentences, alice_persuasion_ads)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


In [8]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc2 = ensemble.RandomForestClassifier()

Y = ads_df['text_source']
X = ads_df.drop(['text_sentence', 'text_source'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size = .5,
                                                    random_state = 0)

In [9]:
rfc2 = rfc2.fit(X_train, y_train)

In [10]:
rfc2.score(X_train, y_train)

0.8256479299899024

### Now trying to see if adding adverbs would help the accuracy:

In [11]:
from collections import Counter

def bag_of_adverbs(text):
    
    ads2 = [token.text
           for token in text
           if token.pos_ == 'ADV'
           if not token.is_stop]
    
    return [item[0] for item in Counter(ads2).most_common(2000)]

In [12]:
def bo_adverbs_features(sentences, common_words):
    
        # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to ads, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.text
                 for token in sentence
                 if (not token.is_stop
                     and token.text in common_words)]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [13]:
# Getting the most common adjectives for each text:
alice_adverbs = bag_of_adverbs(alice_doc)
persuasion_adverbs = bag_of_adverbs(persuasion_doc)

# Combining their counts to avoid duplicates:
alice_persuasion_adverbs = set(alice_adverbs + persuasion_adverbs)

# Getting the count per sentence and making a dataframe:
adverbs_df = bo_adverbs_features(sentences, alice_persuasion_adverbs)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


In [14]:
# Combining the adverb and adjective dfs:
ads_df2 = ads_df.drop(['text_source', 'text_sentence'], axis = 1)
combined_df = pd.concat([ads_df2, adverbs_df], axis = 1)

In [15]:
X2 = combined_df.drop(['text_sentence', 'text_source'], axis = 1)
Y2 = combined_df['text_source']

X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, 
                                                        test_size = .4,
                                                        random_state = 0)

In [16]:
rfc3 = ensemble.RandomForestClassifier()
rfc3.fit(X2_train, Y2_train)

rfc3.score(X2_train, Y2_train)


0.8465638148667601

In [17]:
rfc3.score(X2_test, Y2_test)

0.7253994953742641

## Trying a different text:

In [18]:
sensibility = gutenberg.raw('austen-sense.txt')

In [19]:
pattern = "[\[].*?[\]]"

sensibility = re.sub(pattern, "", sensibility)
sensibility = re.sub(r'CHAPTER \d+', '', sensibility)
sensibility = ' '.join(sensibility.split())

In [20]:
sensibility[0:100]

'The family of Dashwood had long been settled in Sussex. Their estate was large, and their residence '

In [21]:
sensibility_doc = nlp(sensibility)

In [33]:
sensibility_sents = pd.DataFrame([[sents, 'Austen'] for sents in sensibility_doc.sents])

In [34]:
# Most common adverbs and adjectives:

sensibility_ads = bag_of_ads(sensibility_doc)
sensibility_adverbs = bag_of_adverbs(sensibility_doc)

In [35]:
all_ads = set(alice_ads + persuasion_ads + sensibility_ads)
all_adverbs = set(alice_adverbs + persuasion_adverbs + sensibility_adverbs)

In [55]:
ads_df3 = boa_features(sensibility_sents, all_ads)
adverbs_df3 = bo_adverbs_features(sensibility_sents, all_adverbs)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000


In [90]:
# New Text:
# ads_df3 = ads_df3.drop(['text_sentence', 'text_source'], axis = 1)
adverbs_df3 = adverbs_df3.append(ads_df3, ignore_index = True)
combined_df2 = combined_df.append(adverbs_df3, ignore_index = True)

X3 = combined_df2.drop(['text_sentence', 'text_source'], axis = 1)
Y3 = combined_df2['text_source']

ValueError: Plan shapes are not aligned

In [75]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, Y3,
                                                        test_size = .25)

In [76]:
rfc4 = ensemble.RandomForestClassifier()

In [77]:
rfc4.fit(X3_train, y3_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [80]:
rfc4.fit(X3_test, y3_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [81]:
rfc4.score(X3_train, y3_train)

1.0

In [1]:
rfc4.score(X3_test, y3_test)

NameError: name 'rfc4' is not defined